In [28]:
import os

#from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
pd.options.display.max_rows = 10000
pd.options.display.max_columns = 1000
import scipy as sp
import scipy.signal
import scipy.stats

import re

import glob

In [29]:
def BandpassFilter(signal, pass_band):
    b, a = sp.signal.butter(5, pass_band, btype='bandpass', fs=50)
    return sp.signal.filtfilt(b, a, signal)

In [30]:
path = r'C:\Users\aditj\HawkinsLab\Subject1.csv\Copy of all Data\TEST_COPY' # use your path
def load_data(path):
    all_files = glob.glob(path + "/*.csv")

    li = []

    for filename in all_files:
        df = pd.read_csv(filename, index_col=None, header=0)
        li.append(df)

    frame = pd.concat(li, axis=0, ignore_index=True)

    frame.drop("Time", inplace = True, axis=1)
    frame.drop("Resultant", inplace=True, axis=1)
    
    return frame

In [46]:
def processing(df, window_length_s, window_shift_s, fs, df_dict):
    window_length_s = 2
    window_shift_s = 1
    window_length = window_length_s * fs
    window_shift = window_shift_s * fs

    
    for column in df:
        filtered_s =  BandpassFilter(df[column], (1, 7.5))
        df[column] = filtered_s
        datalist = []
        for i in range(0, len(df) - window_length, window_shift):
                window = [df[column][i: i + window_length]]
                #window = pd.Series(window)
                #print(f"window = {window[0].values}")

                if column in df_dict:
                    #print("YES")
                    #print(f"SIZE BEFORE: {len(df_dict[column])}")
                    df_dict[column].append(window[0].values)
                    #print(f"SIZE AFTER: {len(df_dict[column])}")
                else:
                    df_dict[column] = [window[0].values]
                    
    return df_dict

In [41]:
def save_processed_data(df_dict, s1_labels, s2_labels):
    total_labels = np.append(s1_labels, s2_labels)
    total_labels
    df_labels = pd.DataFrame(total_labels)
    df_labels.to_csv(r"C:\Users\aditj\HawkinsLab\Subject1.csv\Copy of all Data\Labels_TOTAL.csv",index=False)
    
    dfX = pd.DataFrame(df_dict['X'])
    dfX.to_csv(r"C:\Users\aditj\HawkinsLab\Subject1.csv\Copy of all Data\X_axis_TOTAL.csv",index=False)
    
    dfY = pd.DataFrame(df_dict['X'])
    dfY.to_csv(r"C:\Users\aditj\HawkinsLab\Subject1.csv\Copy of all Data\Y_axis_TOTAL.csv",index=False)
    
    dfZ = pd.DataFrame(df_dict['Z'])
    dfZ.to_csv(r"C:\Users\aditj\HawkinsLab\Subject1.csv\Copy of all Data\Z_axis_TOTAL.csv",index=False)

In [48]:
def main():
    #Load data for Subject 1
    df1 = load_data(r"C:\Users\aditj\HawkinsLab\Subject1.csv\Copy of all Data\Subject 1 Data")
    
    #Process data for Subject 1
    df_dict = processing(df1, window_length_s = 2, window_shift_s = 1, fs=50, df_dict={})
    
    #create labels for Subject 1
    len_sub1 = len(df_dict['X'])
    s1_labels = np.ones(len_sub1)
    
    #Load data for Subject 2
    df2 = load_data(r"C:\Users\aditj\HawkinsLab\Subject1.csv\Copy of all Data\Subject 2 Data")
    #Process Data for Subject 2
    df_dict = processing(df2, 2, 1, 50, df_dict)
    
    #Create Labels for Subject 2
    len_sub2 = len(df_dict['X']) - len_sub1
    s2_labels = np.ones(len_sub2) * 2
    
    #Save all files 
    save_processed_data(df_dict, s1_labels, s2_labels, )

In [49]:
main()